📝 Translation Model Fine-Tuning with Hugging Face
This notebook demonstrates how to fine-tune a transformer-based sequence-to-sequence model for English-to-French translation using the Hugging Face ecosystem.
We use the opus_books dataset and train a model to generate French translations from English inputs. The process includes preprocessing, training with 🤗 Accelerate, and evaluation using the SacreBLEU metric.

✅ Key Details
Dataset: opus_books (en-fr)

Model: Pretrained Transformer (e.g., BART, T5, etc.)

Training Tool: Hugging Face Accelerate

Metric: SacreBLEU (standard for translation quality)



In [ ]:
# pip install -U datasets huggingface_hub fsspec

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("opus_books", "en-fr")

In [ ]:
raw_datasets

#### Split the dataset for training and testing

In [ ]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

In [ ]:
split_datasets["train"][1]["translation"]

### Import Tokenizer

In [ ]:
from transformers import AutoTokenizer

model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_name, return_tensors="pt")

In [ ]:
# !pip install sacremoses

**Must Need** to ensure that the tokenizer processes the targets in the output language (here, French)

In [ ]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

In [ ]:
max_length = 128 #set maximum length in the sentence because of dealing with seem pretty short


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
# Preperocess with tokenizer
tokenized_dataset = split_datasets.map(preprocess_function, batched=True, remove_columns=split_datasets["train"].column_names,)

### Import model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### Adding Data Collation
Need a data collator to deal with the padding for dynamic batching.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## Matric
BLEU (Bilingual Evaluation Understudy) is a metric that evaluates machine translation by measuring how many words and phrases in the model’s output match the reference translation — it checks n-gram overlap. However, it requires pre-tokenized text, which can lead to inconsistent results across different models and tools.

SacreBLEU is a modern version of BLEU that automatically handles tokenization and standardizes evaluation, making it easier to compare results reliably across models and datasets.

In [ ]:
# !pip install sacrebleu evaluate


In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

### Fine Tuning the model

In [ ]:
from torch.utils.data import DataLoader

tokenized_dataset.set_format("torch")
train_dataloader = DataLoader(
    tokenized_dataset["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_dataset["validation"], collate_fn=data_collator, batch_size=8
)

In [ ]:
# Set optmizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
# Setting Accelerator
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

We use this code to create a linear learning rate scheduler that gradually decreases the learning rate during training to help the model converge more smoothly and avoid overshooting.

In [ ]:
from transformers import get_scheduler

num_train_epochs = 2
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
output_dir = "opus_books-en-to-fr-accelerate"

### Training loop
We are now ready to write the full training loop. To simplify its evaluation part, we define this postprocess() function that takes predictions and labels and converts them to the lists of strings our metric object will expect:

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np


progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)


### Check the Result

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "opus_books-en-to-fr-accelerate"
translator = pipeline("translation", model=model_checkpoint)
translator("My name is Tamal")